# Fast Fourier Transform - Python Implementation

### Algorithm Pseudocode:

### Naive Polynomial Multiplication:
1. Take the coefficient representations of two polynomials (start with two of equal degree)
2. Use two `for` loops to multiply the the coefficients of each polynomial pairwise, giving $n^2$ products
3. Collect like terms
4. Return the output in coefficient representation

Example:

$A(x)=2+3x+7x^2 \implies$ `A=[2,3,7]`

$B(x)=1+2x^2 \implies$ `B=[1,0,2]`

$C(x)=A(x)\cdot B(x)=z+3+11x^2+6x^3+14x^4 \implies$ `C=[2,3,11,6,14]`

$d_A = 2, d_B = 2, d_C=d_B + d_B=4$ where d is the degree of some polynomial

$n_A=3, n_B=3, n_C=d_C+1=5$ where n is the number of terms in some polynomial

We want to add exponents and multiply coefficients. In terms of array indexing, `A[i]*B[j]=C[i+j]` since `i+j` gives us the new exponent, and therefore the index for the coefficent, and `A[i]+B[j]` gives us the coefficient itself at that index.

In [12]:
def naive_multiplication(A: list[int], B: list[int]):
    n_A = len(A)
    n_B = len(B)
    
     d_A = n_A -1
    d_B = n_B -1
    
    # Get the degree and number of terms for the new polynomial, C(x)
    d_C = d_A + d_B
    n_C = d_C + 1
    
    # Initialise an array to store the coefficeints of C(x)
    C = [0 for i in range(n_C)]

    for i in range(n_A):
        for j in range(n_B):
            C[i+j] = C[i+j] + A[i]*B[j]

    return C

A = [2,3,7]
B = [1,0,2]
naive_multiplication(A, B)

[2, 3, 11, 6, 14]

In [17]:
from time import time
from random import randint

A = [randint(0,100000) for _ in range(10000)]
B = [randint(0,100000) for _ in range(10000)]

t = time()
naive_multiplication(A, B)
t = time() - t

print(t)

33.06097459793091


In [71]:
from time import time
from random import randint
from tabulate import tabulate


def test_timing(f):
    n = [10,100,1000,10000]
    times = []
    
    for i in range(len(n)):
        A = [randint(0,999) for _ in range(n[i])]
        B = [randint(0,999) for _ in range(n[i])]
    
        t = time()
        f(A, B)
        t = time() - t
        times.append(t)
        
    table = {"n": n, "time (s)": times}
    print(f"{f.__name__.replace('_',' ')}:")
    print(tabulate(table, headers="keys"))
    
test_timing(naive_multiplication)

# from n=100 to n=1000, x10 more data resulted in x16 runtime
# from n=1000 to n=10,000, x10 more data resulted in x100 runtime

naive multiplication:
    n     time (s)
-----  -----------
   10   0
  100   0.00199175
 1000   0.180518
10000  16.9786


In [78]:
from math import log
from math import exp
from math import pi

def partition_by_parity(P: list[int]):
    even = []
    odd = []
    for i in range(len(P)):
        if i % 2 == 0:
            even.append(P[i])
        else:
            odd.append(P[i])
    return even, odd
        

def FFT(P: list[int]):
    """Convert a list of n numbers, P,
    that represents the coefficients of a polynomial,
    to a point-value representation, using the FFT.
    Return a list of n complex numbers."""
    
    n = len(P)
    
    # Base Case of Recursion
    if n == 1:
        return P
    
#     if log(n, 2).is_integer() == False:
#         return # padding..
    
    # First root of unity
    w = complex(0, exp(2*pi/n))
    
    # Recusion on the paritionted polynomial
    even, odd = partition_by_parity(P)
    y_e, y_o = FFT(even), FFT(odd)
    
    # Setup for calculating values by iteration
    y = [0]*n
    half = int(n/2) 
    
    for j in range(half):
        y[j] = y_e[j] + (w**j)*y_o[j]
        y[j + half] = y_e[j] - (w**j)*y_o[j]
        
    return y
        
print(FFT([1,2,3,4]))

4.810477380965351j
23.140692632779267j
23.140692632779267j
[(10+0j), (-2-9.620954761930703j), (-2+0j), (-2+9.620954761930703j)]
